In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN, SMOTE
from skopt import BayesSearchCV
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from skopt.space import Real, Integer, Categorical
from sklearn.metrics import accuracy_score
import numpy as np # linear algebra
import pandas as pd 

In [ ]:
df = pd.read_csv("FeaturesC/resnet50_features_standardized_l2.csv", index_col = 'video_id')

In [3]:
features = df.values 
y_train = df.index.map(lambda vid: vid[2:-12]).values

X_train = features

In [4]:
X_train.shape

(735, 2048)

In [5]:
# rf = RandomForestClassifier(random_state=42, class_weight="balanced")

# param_space = {
#     'n_estimators': Integer(1, 300),             
#     'max_depth': Integer(1, 20),                  
#     'min_samples_split': Integer(2, 20),          
#     'min_samples_leaf': Integer(1, 20),           
#     'max_features': Categorical(['auto', 'sqrt', 'log2']),
#     'criterion': Categorical(['gini', 'entropy'])   
# }


# bayes_cv = BayesSearchCV(
#     estimator=rf,
#     search_spaces=param_space,
#     n_iter=70, 
#     scoring='accuracy',
#     cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), 
#     n_jobs=-1,  
#     random_state=1,
#     verbose=4
# )

In [6]:
# bayes_cv.fit(X_train, y_train)
# print("Best Parameters:", bayes_cv.best_params_)
# import matplotlib.pyplot as plt
# plt.plot(bayes_cv.cv_results_['mean_test_score'])
# plt.xlabel('Iteration')
# plt.ylabel('Mean F1-Score')
# plt.title('Convergence Plot')
# plt.show()

In [7]:
# print("Best cross-validated accuracy:", bayes_cv.best_score_)

In [ ]:
df_test = pd.read_csv("FeaturesC/resnet50_test_features_standardized_l2.csv", index_col = 'video_id')
X_test = df_test.values

In [9]:
X_test.shape

(294, 2048)

In [10]:
# pred = bayes_cv.predict(X_test)

In [11]:
best_params = {
    'criterion': 'gini',
    'max_depth': 20,
    'max_features': 'auto',  # 'auto' = sqrt(n_features) for classification
    'min_samples_leaf': 6,
    'min_samples_split': 2,
    'n_estimators': 296,
    'random_state': 42  # For reproducibility
}

rf_model = RandomForestClassifier(**best_params)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
pred = rf_model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [12]:
submission_df = pd.DataFrame({
    'ID': df_test.index, 
    'Label': pred
})

# Save the submission file as CSV without an extra index column.
submission_df.to_csv("submission.csv", index=False)